In [4]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     --------------------------------- ------ 51.2/60.9 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 60.9/60.9 kB 804.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
    --------------------------------------- 0.2/11.5 MB 14.3 MB/s eta 0:00:01
   - -------------------------------------- 0.3/11.5 MB 2.5 MB/s eta 0:00:05
   - -------------------------------------- 0.4/11.5 MB 2.3 MB/s eta 0:00:05
   - -------------------------------------- 0.4/11.5 MB 2.1 MB/s eta 0:00:06
   - -------------------------------------- 0.5/11.5 MB 2.0 MB/s eta 0:00:06
   -- ------------------------------------- 0.6/11.5 MB 1.9 MB/s eta 0:00:06
   -- ------------------------------------- 0.7/11.5 MB 1.9 MB/s eta 0:00:06
   -- ------------------------------------- 0.8/11.5 MB 1.8 MB/s eta 0:00:06
   

In [5]:
import pandas as pd

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)  # Increased timeout

driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")

APSRTC = []

def scrape_page():
    # Locate elements  (container)
    routes = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))

    # Loop through each route to extract details
    for route in routes:
        try:
            # Extract route name and link from the route element
            route_name_element = route.find_element(By.XPATH, ".//div[@class='route_details']")
            route_link_element = route.find_element(By.XPATH, ".//a")

            route_name = route_name_element.text
            route_link = route_link_element.get_attribute('href')

            # Append extracted data to list
            APSRTC.append({
                'route_name': route_name,
                'route_link': route_link
            })

        except Exception as e:
            print(f"An error occurred: {e}")
            continue

# Scrape data from the first 5 pages
for page_number in range(1, 6):
    scrape_page()
    if page_number < 5:  # Don't try to click next on the last page
        try:
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Log the action
            print(f"Clicking on page {page_number + 1}")

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Log the successful page navigation
            print(f"Successfully navigated to page {page_number + 1}")

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

# Print the scraped data
for entry in APSRTC:
    print(entry)

# Convert to DataFrame and save to CSV
df = pd.DataFrame(APSRTC)
df.to_csv("apsrtc_details.csv", index=False)

# Close the driver
driver.quit()

df.head()  # Display the first few rows of the DataFrame

Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
{'route_name': 'Vijayawada to Hyderabad\nFrom INR 414', 'route_link': 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad'}
{'route_name': 'Hyderabad to Vijayawada\nFrom INR 367', 'route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada'}
{'route_name': 'Kakinada to Visakhapatnam\nFrom INR 239', 'route_link': 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam'}
{'route_name': 'Visakhapatnam to Kakinada\nFrom INR 239', 'route_link': 'https://www.redbus.in/bus-tickets/visakhapatnam-to-kakinada'}
{'route_name': 'Chittoor (Andhra Pradesh) to Bangalore\nFrom INR 186', 'route_link': 'https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore'}
{'route_name': 'Kadapa to Bangalore\nFrom INR 412', 'route_link': 'https://www.redbus.in/bus-tickets/

,route_name,route_link
0,Vijayawada to Hyderabad\nFrom INR 414,https://www.redbus.in/bus-tickets/vijayawada-t...
1,Hyderabad to Vijayawada\nFrom INR 367,https://www.redbus.in/bus-tickets/hyderabad-to...
2,Kakinada to Visakhapatnam\nFrom INR 239,https://www.redbus.in/bus-tickets/kakinada-to-...
3,Visakhapatnam to Kakinada\nFrom INR 239,https://www.redbus.in/bus-tickets/visakhapatna...
4,Chittoor (Andhra Pradesh) to Bangalore\nFrom I...,https://www.redbus.in/bus-tickets/chittoor-and...
